In [ ]:
import os
from os.path import dirname, basename
from openmm.app import *
from openmm import *
from openmm.unit import kelvin, nanometer, picoseconds
from sys import stdout

In [ ]:
PDB_FILENAME = 'backmapped/A2A/CG/protein/multi_backmapped.pdb'

In [ ]:
pdb = PDBFile(PDB_FILENAME)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addHydrogens(forcefield)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

pdb_folder = dirname(PDB_FILENAME)
minimized_pdb_folder = os.path.join(pdb_folder, 'minimized')
os.makedirs(minimized_pdb_folder, exist_ok=True)

print('Running Minimization...')
simulation.minimizeEnergy(tolerance=1*unit.kilojoules_per_mole/nanometer, maxIterations=10000)

positions = simulation.context.getState(getPositions=True).getPositions()

print('Adding Membrane...')
modeller.addMembrane(forcefield, membraneCenterZ=positions.mean().z)

print('Solvating...')
modeller.addSolvent(forcefield)

system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

print('Saving...')
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(os.path.join(minimized_pdb_folder, basename(PDB_FILENAME)), 'w'))
print('Done')

In [ ]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

simulation_pdb_folder = os.path.join(pdb_folder, 'simulation')
os.makedirs(simulation_pdb_folder, exist_ok=True)

simulation.reporters.append(PDBReporter(os.path.join(simulation_pdb_folder, basename(PDB_FILENAME)), 10000))
simulation.reporters.append(
    StateDataReporter(
        stdout, 10000, step=True,potentialEnergy=True, temperature=True)
)

os.makedirs(os.path.join(simulation_pdb_folder), exist_ok=True)

simulation.minimizeEnergy(tolerance=1*unit.kilojoules_per_mole/nanometer, maxIterations=10000)
simulation.step(50000)